In [1]:
from langchain_community.vectorstores import Chroma
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, Tool

from dotenv import load_dotenv
import os
import fitz
import subprocess

In [2]:
load_dotenv()

True

In [3]:
fire_dir_with_notes="../../../../NOTES/papers/inprogress/Attention-is-all-you-need.pdf"
fire_dir_origin="../../../../NOTES/papers/inprogress/Attentionisallyouneed.pdf"

In [4]:
db_dir = os.path.join(os.getcwd(), "db")
persistent_directory = os.path.join(db_dir, "chroma_db")

In [5]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)  # open pdf file
    text = ""
    for page in doc:  # recursive each page
        text += page.get_text("text") + "\n"  # Extract text and wrap
    return text.strip()

In [6]:
# extract text from pdf
pdf_text=extract_text_from_pdf(fire_dir_origin)

In [7]:
type(pdf_text)

str

In [8]:
def create_vector_store(text:str,metadata:str,chunk_size:int=1000,chunk_overlap:int=100,embedding_model_name:str="text-embedding-3-small"):
    # combine text content and file metadata together
    documents = [Document(page_content=pdf_text, metadata={"source": metadata})]

    # Splits text into chunks
    text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)

    # Define the embedding model
    embeddings = OpenAIEmbeddings(model=embedding_model_name)

    # create chroma db
    db = Chroma.from_documents(
        docs, embeddings, persist_directory=persistent_directory
    )

    return db

In [9]:
db=create_vector_store(text=pdf_text,metadata=fire_dir_with_notes)

Created a chunk of size 2857, which is longer than the specified 1000
Created a chunk of size 4255, which is longer than the specified 1000
Created a chunk of size 1826, which is longer than the specified 1000
Created a chunk of size 2507, which is longer than the specified 1000
Created a chunk of size 3192, which is longer than the specified 1000
Created a chunk of size 3472, which is longer than the specified 1000
Created a chunk of size 3320, which is longer than the specified 1000
Created a chunk of size 3193, which is longer than the specified 1000
Created a chunk of size 2972, which is longer than the specified 1000
Created a chunk of size 3111, which is longer than the specified 1000
Created a chunk of size 3215, which is longer than the specified 1000
Created a chunk of size 3213, which is longer than the specified 1000


In [10]:
# Then use rag / vector store db with llm toghether

In [11]:
# load db

# Define the embedding model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Load create chroma vecstore with the corresponding embedding function
pdf_db = Chroma(persist_directory=persistent_directory,
            embedding_function=embeddings)

/var/folders/5b/b2jjkdb55_g_qzx3lsl5k59m0000gn/T/ipykernel_17071/551135029.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  pdf_db = Chroma(persist_directory=persistent_directory,


In [12]:
# create rag retriever
retriever = db.as_retriever(search_type="similarity",search_kwargs={"k": 1})

In [13]:
llm = ChatOpenAI(model="gpt-4o")

In [14]:
from langchain.memory import ConversationBufferMemory

In [15]:
memory = ConversationBufferMemory(
    memory_key="history", return_messages=True)

/var/folders/5b/b2jjkdb55_g_qzx3lsl5k59m0000gn/T/ipykernel_17071/1976451665.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [16]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder,AIMessagePromptTemplate
from langchain_core.prompts import PromptTemplate

In [17]:
# Identify conversation history based on context and rewrite questions to avoid mis-searching due to short questions

system_message=SystemMessagePromptTemplate.from_template(
    """ Given the following conversation and the input question, rephrase the input question to be a standalone question.

        Provide only ONE action per $JSON_BLOB, as shown:
        ```
        {{
        "action": $TOOL_NAME,
        "action_input": $INPUT
        }}
        ```
        Follow this format:

        Question: input question to answer
        Thought: consider previous and subsequent steps
        Action:
        ```
        $JSON_BLOB
        ```
        Observation: action result
        ... (repeat Thought/Action/Observation N times)
        Thought: I know what to respond
        Action:
        ```
        {{
        "action": "Final Answer",
        "action_input": "Final response to human"
        }}
        ```        
        Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation
     
        """    
)

history_placeholder=MessagesPlaceholder(variable_name="history",optional=True)

human_message=HumanMessagePromptTemplate.from_template(
    """{input}
    
    {agent_scratchpad}
    """
)


contextualize_prompt=ChatPromptTemplate.from_messages([
    system_message,
    history_placeholder,
    human_message
])

In [18]:
# reformulate the question based on chat history
history_aware_retriever = create_history_aware_retriever(
    llm=llm, retriever=retriever, prompt=contextualize_prompt
)

In [19]:
system_message=SystemMessagePromptTemplate.from_template(
    """
    You are an AI assistant. Use the retrieved context below to answer questions.
    If you don't know the answer, just say so.
    {context}
    
    **Answer each question in 3 short sentences. Make the answer concise and clear. Reponse the anwer with plain text.**

    Provide only ONE action per $JSON_BLOB, as shown:
    ```
    {{
    "action": $TOOL_NAME,
    "action_input": $INPUT
    }}
    ```
    Follow this format:

    Question: input question to answer
    Thought: consider previous and subsequent steps
    Action:
    ```
    $JSON_BLOB
    ```
    Observation: action result
    ... (repeat Thought/Action/Observation N times)
    Thought: I know what to respond
    Action:
    ```
    {{
    "action": "Final Answer",
    "action_input": "Final response to human"
    }}
    ```        
    Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation 
    """
)

qa_prompt=ChatPromptTemplate.from_messages([
    system_message,
    history_placeholder,
    human_message
])

In [20]:
# Create a chain for passing a list of Documents to a model.
# feeds all retrieved context into the LLM

qa_chain = create_stuff_documents_chain(llm=llm, prompt=qa_prompt)

In [21]:
# First use history_aware_retriever to adjust the problem (complete historical information)
# Then use retriever to query the knowledge base.
# Finally, the query result is passed to question_answer_chain to generate the final answer.
rag_chain = create_retrieval_chain(retriever=history_aware_retriever, combine_docs_chain=qa_chain)

In [22]:
def open_file(file_path):
    """
    open file on macOS(PDF、Word、TXT)
    :param file_path: the file path
    """
    # make sure the path exists.
    if not os.path.exists(file_path):
        print(f"error: file '{file_path}' does not exist.")
        return
    
    try:
        subprocess.run(["open", file_path], check=True)

        return "Agentfinished, you just need to response the file is opened."
    
    except subprocess.CalledProcessError as e:

        return f"cannot open the file: {e}" 


In [23]:
tools = [
    Tool(
        name="AnswerQuestionWithLocalContent",
        func=lambda input, **kwargs: rag_chain.invoke(
            {"input": input, "history": memory.chat_memory.messages,"agent_scratchpad":""}
        ),
        description="Use RAG to answer questions based on retrieved context on local laptop. Use this tool to answer the corresponding file's local path.",
    ),
    Tool(
        name="OpenLocalPath",
        func=open_file,
        description="Use this tool to open the local file."
    )
]


In [24]:
from langchain.agents import AgentExecutor, create_react_agent

In [25]:
template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
# When calling a tool, always use the following format:

# ```
# {{
#     "action": "ConcatenateStrings",
#     "action_input": {{
#     "a": "<first_string>",
#     "b": "<second_string>",
#     ...
#     }}
# }}
# ```
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

In [26]:
prompt_template=PromptTemplate.from_template(template)

In [27]:
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt_template,
    stop_sequence=True,
)

In [28]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,  # The agent to execute
    tools=tools,  # List of tools available to the agent
    verbose=True,  # Enable verbose logging
    memory=memory,
    handle_parsing_errors=True,  # Handle parsing errors gracefully
)

In [29]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [30]:
input="What is transformer, answer it with local knowledge."
response = agent_executor.invoke({"input": input})
memory.chat_memory.add_message(HumanMessage(content=input))
memory.chat_memory.add_message(AIMessage(content=response["output"]))



> Entering new AgentExecutor chain...
To answer the question about what a transformer is using local knowledge, I will use the AnswerQuestionWithLocalContent tool to query the relevant local content.

Action: AnswerQuestionWithLocalContent
Action Input: "What is transformer?" 
{'input': 'What is transformer?" \n', 'history': [], 'agent_scratchpad': '', 'context': [Document(metadata={'source': '../../../../NOTES/papers/inprogress/Attention-is-all-you-need.pdf'}, page_content='Figure 1: The Transformer - model architecture.\nThe Transformer follows this overall architecture using stacked self-attention and point-wise, fully\nconnected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,\nrespectively.\n3.1\nEncoder and Decoder Stacks\nEncoder:\nThe encoder is composed of a stack of N = 6 identical layers. Each layer has two\nsub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-\nwise fully connected fee

In [31]:
response

{'input': 'What is transformer, answer it with local knowledge.',
 'history': [HumanMessage(content='What is transformer, answer it with local knowledge.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='A Transformer is a model architecture utilizing self-attention and fully connected layers for encoders and decoders, crucial in tasks like machine translation.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is transformer, answer it with local knowledge.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='A Transformer is a model architecture utilizing self-attention and fully connected layers for encoders and decoders, crucial in tasks like machine translation.', additional_kwargs={}, response_metadata={})],
 'output': 'A Transformer is a model architecture utilizing self-attention and fully connected layers for encoders and decoders, crucial in tasks like machine translation.'}

In [32]:
response['output']

'A Transformer is a model architecture utilizing self-attention and fully connected layers for encoders and decoders, crucial in tasks like machine translation.'

In [33]:
response = agent_executor.invoke({"input": "Where is the transformer paper exist on my laptop?"})
memory.chat_memory.add_message(HumanMessage(content=input))
memory.chat_memory.add_message(AIMessage(content=response["output"]))



> Entering new AgentExecutor chain...
I should search for the transformer paper in the local files to see where it exists.
Action: AnswerQuestionWithLocalContent
Action Input: "transformer paper location on laptop"{'input': 'transformer paper location on laptop', 'history': [HumanMessage(content='What is transformer, answer it with local knowledge.', additional_kwargs={}, response_metadata={}), AIMessage(content='A Transformer is a model architecture utilizing self-attention and fully connected layers for encoders and decoders, crucial in tasks like machine translation.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is transformer, answer it with local knowledge.', additional_kwargs={}, response_metadata={}), AIMessage(content='A Transformer is a model architecture utilizing self-attention and fully connected layers for encoders and decoders, crucial in tasks like machine translation.', additional_kwargs={}, response_metadata={})], 'agent_scratchpad': '', '

In [34]:
response['output']

'The Transformer paper is located at the following path on your laptop: `../../../../NOTES/papers/inprogress/Attention-is-all-you-need.pdf`.'

In [37]:
# response = agent_executor.invoke({"input": "help me open the file."})
# memory.chat_memory.add_message(HumanMessage(content=input))
# memory.chat_memory.add_message(AIMessage(content=response["output"]))

In [ ]:
# response['output']